# GreatKingdom 7x7 Human vs AI (Colab)

Text-based play (no GUI). Run cells top to bottom.


In [1]:
# --- Repo path ---
# Colab: set to the cloned repo path, e.g. /content/GreatKingdom_zero
# Local: leave empty to auto-detect repo root from cwd.
REPO_DIR = ''

import os, sys
cwd = os.getcwd()
repo = REPO_DIR if REPO_DIR else cwd
if os.path.basename(repo).lower() == 'notebooks':
    repo = os.path.dirname(repo)

if not os.path.isdir(repo):
    raise FileNotFoundError(f'Repo not found at: {repo}')

if repo not in sys.path:
    sys.path.insert(0, repo)
REPO = repo
print('Repo:', REPO)


Repo: c:\Users\sangmin\Desktop\python_projects\1.Active\GreatKingdom_zero


In [2]:
# --- Dependencies ---
import importlib.util, subprocess, sys

def _ensure(pkg):
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

_ensure('numpy')
_ensure('scipy')

import numpy as np
import torch


In [7]:
# --- Parameters ---
import os
BOARD_SIZE = 7
CENTER_WALL = True
KOMI = 3  # choose 3 or 4
N_SIM = 100  # MCTS simulations per move
HUMAN_COLOR = 'black'  # 'black' or 'white'

# Model features (7x7 profile defaults)
USE_LIBERTY_FEATURES = True
LIBERTY_BINS = 1
USE_LAST_MOVES = False

C_PUCT = 2.0
DIRICHLET_ALPHA = 0.3
DIRICHLET_EPSILON = 0.25

CKPT_PATH = os.path.join(REPO, 'checkpoints', 'board_7', 'center_wall_on', 'alphazero_best.pt')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)
print('Checkpoint:', CKPT_PATH)


Device: cpu
Checkpoint: c:\Users\sangmin\Desktop\python_projects\1.Active\GreatKingdom_zero\checkpoints\board_7\center_wall_on\alphazero_best.pt


In [8]:
# --- Load model and create MCTS ---
from env.env import GreatKingdomEnv
from network import AlphaZeroNetwork
from mcts_alphazero import AlphaZeroMCTS
from game_result import winner_label_from_step

env = GreatKingdomEnv(board_size=BOARD_SIZE, center_wall=CENTER_WALL, komi=KOMI)

import os
if not os.path.exists(CKPT_PATH):
    raise FileNotFoundError(f'Checkpoint not found: {CKPT_PATH}')

ckpt = torch.load(CKPT_PATH, map_location=DEVICE, weights_only=False)
state_dict = ckpt['network']

# Infer num_res_blocks and num_channels from checkpoint
policy_w = state_dict.get('policy_conv.weight')
num_channels = int(policy_w.shape[1]) if policy_w is not None else 64
res_blocks = [k for k in state_dict.keys() if k.startswith('res_blocks.') and k.endswith('.conv1.weight')]
num_res_blocks = len(res_blocks) if res_blocks else 3

net = AlphaZeroNetwork(
    board_size=BOARD_SIZE,
    num_res_blocks=num_res_blocks,
    num_channels=num_channels,
    use_liberty_features=USE_LIBERTY_FEATURES,
    liberty_bins=LIBERTY_BINS,
    use_last_moves=USE_LAST_MOVES
).to(DEVICE)

net.load_state_dict(state_dict)
net.eval()

mcts = AlphaZeroMCTS(
    net, env,
    num_simulations=N_SIM,
    c_puct=C_PUCT,
    dirichlet_alpha=DIRICHLET_ALPHA,
    dirichlet_epsilon=DIRICHLET_EPSILON,
    eval_batch_size=1,
    use_liberty_features=USE_LIBERTY_FEATURES,
    liberty_bins=LIBERTY_BINS,
    use_last_moves=USE_LAST_MOVES
)

print('Model:', num_res_blocks, 'res blocks |', num_channels, 'channels')


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy._core.multiarray.scalar was not an allowed global by default. Please use `torch.serialization.add_safe_globals([numpy._core.multiarray.scalar])` or the `torch.serialization.safe_globals([numpy._core.multiarray.scalar])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn 

In [ ]:
# --- Simple ASCII renderer ---
def render_ascii(env):
    symbols = {0: '.', 1: 'B', 2: 'W', 3: '#'}
    header = '  ' + ' '.join(chr(ord('A') + i) for i in range(env.board_size))
    print(header)
    for r in range(env.board_size):
        row = ' '.join(symbols[int(v)] for v in env.board[r])
        print(f'{r+1} {row}')
    print('Turn:', 'Black' if env.current_player == 1 else 'White')
    scores = env.get_territory_scores()
    print(f"Territory - Black: {scores['black']}, White: {scores['white']}")
    print('-' * 20)

def parse_move(text, board_size):
    text = text.strip().lower()
    if text in ('p', 'pass'):
        return board_size * board_size

    # A1-style: e.g., a1, d5
    if len(text) >= 2 and text[0].isalpha():
        col = ord(text[0]) - ord('a')
        try:
            row = int(text[1:]) - 1
        except ValueError:
            return None
        if 0 <= row < board_size and 0 <= col < board_size:
            return row * board_size + col
        return None

    # Space-separated: r c
    parts = text.split()
    if len(parts) != 2:
        return None
    try:
        r = int(parts[0])
        c = int(parts[1])
    except ValueError:
        return None
    if r < 0 or c < 0 or r >= board_size or c >= board_size:
        return None
    return r * board_size + c


In [ ]:
# --- Game loop ---
env.reset()
last_moves = (None, None) if USE_LAST_MOVES else None
human_player = 1 if HUMAN_COLOR == 'black' else 2

while True:
    render_ascii(env)

    if env.current_player == human_player:
        legal = env.get_legal_moves()
        move = None
        while move is None:
            raw = input('Your move (r c / a1 / pass): ')
            action = parse_move(raw, env.board_size)
            if action is None:
                print('Invalid input. Example: 2 3 or a1 or pass')
                continue
            if not legal[action]:
                print('Illegal move. Try again.')
                continue
            move = action
        action = move
    else:
        if USE_LAST_MOVES:
            state = (env.board.copy(), env.current_player, env.consecutive_passes, last_moves)
        else:
            state = (env.board.copy(), env.current_player, env.consecutive_passes)
        action, _ = mcts.run(state, temperature=0.0, add_root_noise=False)
        print('AI move:', 'pass' if action == env.pass_action else divmod(action, env.board_size))

    _, reward, done, info = env.step(action)
    if USE_LAST_MOVES and action != env.pass_action:
        last_moves = (action, last_moves[0])

    if done:
        render_ascii(env)
        winner = winner_label_from_step(info, reward, env.current_player)
        print('Game Over:', info.get('result', ''), '| Winner:', winner)
        break
